In [1]:
!pip install tensorflow transformers


In [2]:
import pandas as pd
import numpy as np
import gdown
import requests
import plotly.express as px
import plotly.figure_factory as ff
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score



# Objetivo da atividade
O objetivo desta atividade é treinar um modelo de rede neural usando a arquitetura BERT (Bidirecional Encoder Representations from Transformers) para detectar contas de bots no Twitter.

1. Preparação dos Dados : Carregar e pré-processar um conjunto de dados que contém tweets, rótulos que indicam se a conta é um bot ou não, e outros atributos relevantes.
2. Treinamento do Modelo : Utilização do BERT para aprender a identificar padrões nos tweets que podem indicar se uma conta é um bot. O modelo é treinado em um conjunto de dados rotulados, onde as contas são definidas como bot (1) ou não bot (0).
3. Avaliação do Modelo : Após o treinamento, o modelo é testado em um conjunto de dados separados para avaliar sua eficácia em prever corretamente a classe (bot ou não bot) de novas contas com base nos tweets.

In [3]:
arquivo_destino_base = "dataset_{}.csv"

id = {
    "falecidoTwitter": "11xP5JEC3Y4YoNlkwKwfWyHOTQWhxp7zn"
}

dataframes = {}

for key, file_id in id.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    # Baixar o arquivo
    gdown.download(url, arquivo_destino, quiet=False)

    # Ler o arquivo CSV
    df = pd.read_csv(arquivo_destino, sep=",")
    dataframes[key] = df

print(dataframes)

Downloading...
From: https://drive.google.com/uc?id=11xP5JEC3Y4YoNlkwKwfWyHOTQWhxp7zn
To: /content/dataset_falecidoTwitter.csv
100%|██████████| 7.46M/7.46M [00:00<00:00, 43.4MB/s]


{'falecidoTwitter':        User ID         Username  \
0       132131            flong   
1       289683   hinesstephanie   
2       779715       roberttran   
3       696168           pmason   
4       704441           noah87   
...        ...              ...   
49995   491196            uberg   
49996   739297     jessicamunoz   
49997   674475   lynncunningham   
49998   167081  richardthompson   
49999   311204         daniel29   

                                                   Tweet  Retweet Count  \
0      Station activity person against natural majori...             85   
1      Authority research natural life material staff...             55   
2      Manage whose quickly especially foot none to g...              6   
3      Just cover eight opportunity strong policy which.             54   
4                          Animal sign six data good or.             26   
...                                                  ...            ...   
49995  Want but put card direction

# Análise estatítica / exploratória

In [4]:
df = pd.read_csv('dataset_falecidoTwitter.csv')
df

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention
...,...,...,...,...,...,...,...,...,...,...,...
49995,491196,uberg,Want but put card direction know miss former h...,64,0,9911,True,1,Lake Kimberlyburgh,2023-04-20 11:06:26,teach quality ten education any
49996,739297,jessicamunoz,Provide whole maybe agree church respond most ...,18,5,9900,False,1,Greenbury,2022-10-18 03:57:35,add walk among believe
49997,674475,lynncunningham,Bring different everyone international capital...,43,3,6313,True,1,Deborahfort,2020-07-08 03:54:08,onto admit artist first
49998,167081,richardthompson,Than about single generation itself seek sell ...,45,1,6343,False,0,Stephenside,2022-03-22 12:13:44,star


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   User ID         50000 non-null  int64 
 1   Username        50000 non-null  object
 2   Tweet           50000 non-null  object
 3   Retweet Count   50000 non-null  int64 
 4   Mention Count   50000 non-null  int64 
 5   Follower Count  50000 non-null  int64 
 6   Verified        50000 non-null  bool  
 7   Bot Label       50000 non-null  int64 
 8   Location        50000 non-null  object
 9   Created At      50000 non-null  object
 10  Hashtags        41659 non-null  object
dtypes: bool(1), int64(5), object(5)
memory usage: 3.9+ MB
None


In [ ]:
# Verificar valores nulos no dataset
print(df.isnull().sum())


User ID              0
Username             0
Tweet                0
Retweet Count        0
Mention Count        0
Follower Count       0
Verified             0
Bot Label            0
Location             0
Created At           0
Hashtags          8341
dtype: int64


## Distribuição de Bots vs Usuários Legítimos
A presença de um número igual de bots e usuários legítimos pode ser benéfica para o treinamento do modelo. Um conjunto de dados equilibrado ajuda a evitar o risco, onde o modelo pode oferecer a pré-visualização da classe com mais exemplos (neste caso, se houvesse mais usuários legítimos do que bots).

In [ ]:
# Gráfico de barras para a distribuição de Bots vs Usuários Legítimos
fig = px.histogram(df, x='Bot Label', title='Distribuição de Bots e Usuários Legítimos',
                   labels={'Bot Label':'Bot (1) ou Não (0)'},
                   color='Bot Label',
                   barmode='group')
fig.show()

In [ ]:
# Estatísticas descritivas das colunas numéricas
print(df[['Retweet Count', 'Mention Count', 'Follower Count']].describe())


       Retweet Count  Mention Count  Follower Count
count    50000.00000   50000.000000    50000.000000
mean        50.00560       2.513760     4988.602380
std         29.18116       1.708563     2878.742898
min          0.00000       0.000000        0.000000
25%         25.00000       1.000000     2487.750000
50%         50.00000       3.000000     4991.500000
75%         75.00000       4.000000     7471.000000
max        100.00000       5.000000    10000.000000


## Contas verificadas vs bots
 O fato de haver mais contas verificadas do que bots pode indicar que as contas verificadas tendem a ser mais autênticas e reconhecidas, uma vez que a verificação geralmente é concedida a indivíduos ou organizações que tenham relevância pública. Isso sugere que usuários legítimos têm uma representação significativa na plataforma, o que é um sinal positivo em relação à integridade da rede social.

In [ ]:
# Gráfico de barras mostrando contas verificadas vs bots
fig = px.histogram(df, x='Verified', color='Bot Label', barmode='group',
                   title='Contas Verificadas vs Bot',
                   labels={'Bot Label':'Bot (1) vs Não Bot (0)', 'Verified':'Verificado (True/False)'})
fig.show()


## Matriz de Correlação
1. Correlação Forte:
  ‘Retweet Count’ e ‘Mention Count’ podem ter uma correlação alta, indicando que tweets com mais retweets tendem a ter mais menções.
  ‘Follower Count’ pode ter uma correlação significativa com ‘Retweet Count’ e ‘Mention Count’, sugerindo que usuários com mais seguidores tendem a ter tweets mais retweetados e mencionados.
2. Correlação Moderada:
  ‘Hashtag Count’ pode ter uma correlação moderada com ‘Retweet Count’ e ‘Mention Count’, indicando que o uso de hashtags pode influenciar a visibilidade e engajamento dos tweets.
3. Correlação Fraca ou Negativa:
  ‘Bot Label’ pode ter correlações variadas com as outras variáveis. Por exemplo, contas rotuladas como bots podem ter padrões diferentes de retweets, menções e seguidores em comparação com contas não rotuladas como bots.
4. Autocorrelação:
  Cada variável terá uma correlação perfeita consigo mesma (1.0), o que é esperado.

In [ ]:
# Calcular correlação
correlation = df[['Retweet Count', 'Mention Count', 'Follower Count', 'Hashtag Count', 'Bot Label']].corr()

# Heatmap da correlação usando Plotly
fig = ff.create_annotated_heatmap(
    z=correlation.values,
    x=list(correlation.columns),
    y=list(correlation.index),
    annotation_text=correlation.round(2).values,
    colorscale='Viridis',
    showscale=True)

fig.update_layout(title='Matriz de Correlação')
fig.show()


# Rede Neural Bert
BERT é um modelo de aprendizagem profundo desenvolvido pelo Google em 2018 que revolucionou o processamento de linguagem natural (PNL). Ele se baseia na arquitetura de transformadores e é projetado para entender o contexto de palavras em uma frase, considerando tanto as palavras que vêm antes quanto as que vêm depois de uma palavra-alvo.

Características Principais do BERT:

    Representações Bidirecionais : Ao contrário dos modelos anteriores, que processavam o texto de forma unidirecional (da esquerda para a direita ou vice-versa), o BERT analisa as palavras em uma sequência de maneira bidirecional. Isso permite que o modelo compreenda melhor o contexto completo em que uma palavra aparece.

    Pré-treinamento : O BERT é inicialmente treinado em grandes corpora de texto usando tarefas como a previsão de palavras mascaradas (Masked Language Model) e a classificação de pares de frases (Next Sentence Prediction).

    Ajuste Fino : Após o pré-treinamento, o modelo pode ser ajustado para tarefas específicas, como classificação de texto, análise de sentimentos ou, no seu caso, detecção de bots. Durante o ajuste fino, o modelo aprende a aplicar seu conhecimento antecipadamente ao novo conjunto de dados.

    Uso de Embeddings : O BERT gera embeddings (representações variadas) para palavras e frases que capturam relações semânticas e contextuais. Isso é útil em tarefas de PNL, onde o significado pode depender do contexto.

    Multi-Tarefa : O modelo pode ser utilizado em diversas tarefas de PNL sem a necessidade de retreinamento completo, bastando ajustar a última camada para a tarefa desejada.

In [13]:

# Reduzir o tamanho do conjunto de dados
sample_size = 500  # Use um número pequeno para acelerar o treinamento
df_small = df.sample(n=sample_size, random_state=42)

# Dividir em dados de entrada (X) e rótulos (y)
X = df_small['Tweet']
y = df_small['Bot Label']

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Carregar o tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizar e codificar os dados de treinamento e teste
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, return_tensors='tf', max_length=64)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, return_tensors='tf', max_length=64)

# Converter os dados de treinamento e teste para o formato TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).shuffle(1000).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

# Inicializar o modelo BERT pré-treinado
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compilar o modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
bert_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Treinar o modelo
bert_model.fit(train_dataset, epochs=2)

# Fazer previsões
y_pred_prob = bert_model.predict(test_dataset)
y_pred = np.argmax(y_pred_prob.logits, axis=1)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
25/25 [==============================] - 157s 4s/step - loss: 0.7078 - accuracy: 0.5100
Epoch 2/2
7/7 [==============================] - 9s 729ms/step
Accuracy: 0.4200
Precision: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


INDO AO INFINITO E ALÉM

In [14]:
# Exemplo de previsão
tweet_example = ["Estou com saudade de twittar e reclamar da vida, portanto, farei desse tweet a minha lamentação sobre amanhã JÁ ser segunda e o final de semana ter passado CORRENDO."]
inputs = tokenizer(tweet_example, return_tensors='tf', padding=True, truncation=True, max_length=128)
predictions = model(inputs)

predicted_label = tf.argmax(predictions.logits, axis=-1).numpy()
print(f'Predicted label: {predicted_label[0]}')

Predicted label: 0
